<a href="https://colab.research.google.com/github/alechacon99/INST767-SP2023/blob/main/Alejandro_Chacon_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3

In the assignment, you are asked to calculate the PMI of words in the Shakespeare collection.

PMI is defined as the follows:

$$PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}$$

The larger the magnitude of PMI for $x$ and $y$ is, the more information you know about the probability of seeing $y$ having just seen $x$ (and vice-versa, since PMI is symmetrical). If seeing $x$ gives you no information about seeing $y$, then $x$ and $y$ are independent and the PMI is zero. You may want to check out this classic paper [Word association norms, mutual information, and lexicography](https://aclanthology.org/J90-1003.pdf) if you'd like a more detailed discussion on the use of PMI in NLP.

For this assignment, $p(x)$ is the probability of seeing word $x$, and it should be calculated by deviding the word count of $x$ by the total word count. $p(x,y)$ is the probability of seeing word $x$ and $y$ cooccurring in the same line, it should be calculated by deviding the cooccurrence of the two words by the total word count. We provided an example PMI calculation function for you.

As discussed in the lecture session, please write two separate implementations, one using the broadcasting mechanism in Spark, and the other using join. Please save your result in `/content/pmi_broadcast` (for broadcasting implementation) and `/content/pmi_join` (for using join). 

**Please pay attention to the following specifications:**

1. In addition to PMI, it's also important to know how many times the pair actually co-occurs, so you'll include this information in the output. Each each line should contain four elements, seperated by a tab char ("\t")

  - word1, 
  - word2, 
  - cooccurrence of the two words, 
  - PMI of the two words. 
2. To reduce the number of spurious pairs, please filter out pairs that occur few times. Please use a threshold of 10, that is, we are only interested in pairs of words that co-occur in ten or more lines.
3. To submit, download your notebook as a ipynb file and uploaded it to ELMS.

# Installing Spark


Downloading Apache Spark from a repository, unzipping, and installing it. This step may take up to 10 mins to run. So please be patient.


In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


Setting up environment 

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext, SparkConf
sc = SparkContext.getOrCreate()

# Loading Shakespeare Dataset

Here we load the Shakespeare dataset as a RDD for you.

We have also provided some "helper" codes.

In [ ]:
!wget https://www.gutenberg.org/cache/epub/100/pg100.txt
!mv pg100.txt /content/shakespeare.txt

In [ ]:
shakespeare = sc.textFile('/content/shakespeare.txt')

In [ ]:
import re
PAT = re.compile("(^[^a-z]+|[^a-z]+$)")

def tokenize(line):
    #tokenize the string, remove unnecessary characters
    tokens = [re.sub(PAT, "", t.lower()) for t in line.split()]
    #returning non-empty strings as tokenization result
    return list(filter(lambda x:len(x)>0, tokens))

In [ ]:
# feel free to use the following sentences to debug.
sentences = ['a fox jumps over the lazy dog',
             'fox jumps over the lazy dog',
             'jumps over the lazy dog',
             '',
             '',
             'i really really like spark',
             'really really']

In [ ]:
# This prepares the word count that you are familiar with.
word_freq = shakespeare.flatMap(tokenize) \
          .map(lambda word: (word, 1)) \
          .reduceByKey(lambda a, b: a + b)

In [ ]:
def get_pairs(tokens):
  for i in range(len(tokens)):
    for j in range(i+1, len(tokens)):
      # we are not interested the coocurrence of the same word.
      if tokens[i] == tokens[j]:
        continue
      yield (tokens[i], tokens[j])
      yield (tokens[j], tokens[i])

In [ ]:
# This cells prepares the cooccurrence count, which we have demonstrated in class.
pairs = shakespeare.map(tokenize).flatMap(lambda x:get_pairs(x))
pairs_count = pairs.map(lambda x:(x,1)).reduceByKey(lambda a, b: a + b)

In [ ]:
# This cell provides a sample implementation of PMI calculation, feel free to use this.
# Note that word_count_total is the count of all tokens. Since this is a integer, there is no need to broadcast it.
import math

def calc_pmi(word1_count, word2_count, word12_cooccur, word_count_total):
  fx = word1_count / word_count_total
  fy = word2_count / word_count_total
  fxy = word12_cooccur / word_count_total
  return math.log(fxy / (fx * fy), 10)

# Calculate PMI using Broadcast [4 points]
Let's start by calculating PMI using the broadcasting mechanism of Spark. We have write a few lines of code to get you started

In [ ]:
from collections import Counter
word_freq_counter = Counter()

In [ ]:
for word, count in word_freq.collect():
  word_freq_counter[word] = count

In [ ]:
total_wc = sum(word_freq_counter.values())

In [ ]:
word_freq_counter_bc = sc.broadcast(word_freq_counter) # bc = broadcast

Please write your codes below. Feel free to add more cells as you wish.

pairs counr's tuple looks like ((A, B), n)

find n_A, n_B in word_freq_counter_bc.value by looking up x[0][0], x[0][1]

In [20]:
pairs_pmi = pairs_count.map(lambda x : (x[0], x[1], word_freq_counter_bc.value(x[0][0]), word_freq_counter_bc.value(x[0][1]))) \
  .map(lambda y: f'{y[0][0]}\t{y[0][1]}\t{y[1][0]}\t{calc_pmi(y[1][2], y[1][3], y[1][2], total_wc)}') \
  .filter(lambda z: z[3] >= 10)

## y looks like (A, B), n, n_A, n_B using y....
## for desired format consider 


In [ ]:
pairs_pmi.collect()

# Calculate PMI using join [4 points]

Please write your codes below. Feel free to add more cells as you wish.



pairs_count (A, B), n 
word_freq (A, n_A)

((A, B), n) -> (A, (B, n)) -> ((A, B), n, n_)
((A< B), n) -> x[0] is (A, B), x[1] is n,
A <- x[0][0]
B <- x[0][1]
n <- x[1]
(a, (B,n)) is (x[0][0], (x[0][1], x[1]))

(A, (B, n)), (A, n_A) ----join----> (A, ((B, n), n_A)

map again to get ((A, B), n, n_A) or maybe map to (B, (A, n, n_A))?

(B, (A, n, n_A)), (B, n_B) ----join----> (B, ((A, n, n_A), n_B)

final map to ((A, B), n, n_A, n_B)

# Answer a few questions with your result [2 points]

1. How many distinct PMI pairs did you extract?

2. What's the pair (x, y) (or pairs if there are ties) with the highest PMI? Use Similarly, what's the pair with the lowest (negative) PMI? Do you find the results make sense? Write a sentence or two to explain why or why not.

You can compute the answer to the two questions however you wish and you're not required to use Spark.